# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [98]:
from pstats import Stats
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [99]:
from iex_secrets import IEX_CLOUD_API_TOKEN

stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [100]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 75507180,
 'calculationPrice': 'close',
 'change': -1.3,
 'changePercent': -0.00847,
 'close': 157.26,
 'closeSource': 'fcaoiifl',
 'closeTime': 1718282350313,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 157.07,
 'delayedPriceTime': 1700031611822,
 'extendedChange': -0.16,
 'extendedChangePercent': -0.00107,
 'extendedPrice': 158.9,
 'extendedPriceTime': 1738563809810,
 'high': 157.26,
 'highSource': 'pde ictmereia1nyd5l ue ',
 'highTime': 1715738779226,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 158.97,
 'iexCloseTime': 1684560114015,
 'iexLastUpdated': 1723866943154,
 'iexMarketPercent': 0.02040903469293076,
 'iexOpen': 162.83,
 'iexOpenTime': 1721325326132,
 'iexRealtimePrice': 160.94,
 'iexRealtimeSize': 101,
 'iexVolume': 1402320,
 'lastTradeTime': 1681340125719,
 'latestPrice': 160.8,
 'latestSource': 'Close',
 'latestTime': 'July 22, 2022',
 'latestUpdate': 1685528555259,
 'latestVolume': 696414

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [101]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

25.45

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [102]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [103]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?token={IEX_CLOUD_API_TOKEN}&types=quote&symbols={symbol_string}'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in data:
        # print(data[symbol]['quote']['latestPrice'])
        # print(data[symbol]['quote']['peRatio'])
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                ],
                index = my_columns,
            ),
            ignore_index = True
        )
final_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch?token=Tpk_0020df4fd968417db3419f6708aba4fa&types=quote&symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
https://sandbox.iexapis.com/stable/stock/market/batch?token=Tpk_0020df4fd968417db3419f6708aba4fa&types=quote&symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,F

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,126.95,30.31,N/A
1,AAL,13.79,-3.74,N/A
2,AAP,195.29,21.53,N/A
3,AAPL,157.40,26.13,N/A
4,ABBV,153.01,21.5,N/A
...,...,...,...,...
496,YUM,122.09,21.92,N/A
497,ZBH,108.00,107.66,N/A
498,ZBRA,337.58,21.62,N/A
499,ZION,52.93,5.66,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [104]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,109.20,0.99,N/A
1,NRG,36.31,2.27,N/A
2,EBAY,46.79,2.64,N/A
3,AMZN,128.13,2.97,N/A
4,COF,113.78,4.28,N/A
5,NUE,123.26,4.48,N/A
6,PVH,64.86,4.57,N/A
7,SYF,33.34,4.62,N/A
8,F,12.85,4.63,N/A
9,KSS,30.66,4.64,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [105]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [106]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [107]:
position_size = math.floor(float(portfolio_size)/len(final_dataframe.index))
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(float(position_size)/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,109.20,0.99,183
1,NRG,36.31,2.27,550
2,EBAY,46.79,2.64,427
3,AMZN,128.13,2.97,156
4,COF,113.78,4.28,175
5,NUE,123.26,4.48,162
6,PVH,64.86,4.57,308
7,SYF,33.34,4.62,599
8,F,12.85,4.63,1556
9,KSS,30.66,4.64,652


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [108]:
symbol = 'AAPL'
batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?token={IEX_CLOUD_API_TOKEN}&types=quote,advanced-stats&symbols={symbol}'
data = requests.get(batch_api_url).json()
# print(data['AAPL']['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda
# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_GP = enterprise_value / gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [109]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in data:
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [110]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.81,N/A,30.2,N/A,7.66,N/A,5.82,N/A,21.651813,N/A,11.243450,N/A,N/A
1,AAL,14.32,N/A,-3.79,N/A,-1.02,N/A,0.272,N/A,-22.226296,N/A,0.994139,N/A,N/A
2,AAP,199.80,N/A,21.74,N/A,4.15,N/A,1.04,N/A,11.912512,N/A,2.579941,N/A,N/A
3,AAPL,155.28,N/A,25.32,N/A,37.58,N/A,6.81,N/A,19.959844,N/A,15.473248,N/A,N/A
4,ABBV,149.88,N/A,21.54,N/A,16.76,N/A,4.6,N/A,11.722570,N/A,8.153973,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.00,N/A,22.63,N/A,-4.12,N/A,5.32,N/A,20.378249,N/A,9.277969,N/A,N/A
497,ZBH,109.00,N/A,110.96,N/A,1.94,N/A,3.04,N/A,18.514937,N/A,5.472932,N/A,N/A
498,ZBRA,327.63,N/A,21.54,N/A,6,N/A,3.09,N/A,16.809428,N/A,7.188635,N/A,N/A
499,ZION,53.87,N/A,5.71,N/A,1.39,N/A,2.4,N/A,4.527025,N/A,2.244148,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [111]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.800,N/A,61.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,289.310,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,295.280,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.050,N/A,50.33,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.930,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.060,N/A,14.31,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,60.080,N/A,32.99,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,32.920,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,103.670,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [113]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.81,N/A,30.20,N/A,7.66,N/A,5.820,N/A,21.651813,N/A,11.243450,N/A,N/A
1,AAL,14.32,N/A,-3.79,N/A,-1.02,N/A,0.272,N/A,-22.226296,N/A,0.994139,N/A,N/A
2,AAP,199.80,N/A,21.74,N/A,4.15,N/A,1.040,N/A,11.912512,N/A,2.579941,N/A,N/A
3,AAPL,155.28,N/A,25.32,N/A,37.58,N/A,6.810,N/A,19.959844,N/A,15.473248,N/A,N/A
4,ABBV,149.88,N/A,21.54,N/A,16.76,N/A,4.600,N/A,11.722570,N/A,8.153973,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.00,N/A,22.63,N/A,-4.12,N/A,5.320,N/A,20.378249,N/A,9.277969,N/A,N/A
497,ZBH,109.00,N/A,110.96,N/A,1.94,N/A,3.040,N/A,18.514937,N/A,5.472932,N/A,N/A
498,ZBRA,327.63,N/A,21.54,N/A,6.00,N/A,3.090,N/A,16.809428,N/A,7.188635,N/A,N/A
499,ZION,53.87,N/A,5.71,N/A,1.39,N/A,2.400,N/A,4.527025,N/A,2.244148,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [114]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [129]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
rv_dataframe

/Users/luxiaotong/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,451.65,44,411.08,1.0,24.16,0.86,15.37,0.86,125.928185,1.0,18.348054,0.72,97.644711
1,PAYC,336.95,59,85.53,0.8,20.41,0.8,17.89,0.94,52.361429,0.94,18.870756,0.76,97.045908
2,CDNS,171.51,116,64.84,0.76,17.54,0.73,15.23,0.84,50.793766,0.92,16.694200,0.6,95.508982
3,NVDA,177.24,112,48.02,0.56,17.54,0.73,15.77,0.88,36.255665,0.78,23.216891,0.92,95.309381
4,IDXX,411.82,48,47.74,0.54,54.10,0.94,10.73,0.54,33.139104,0.7,18.641545,0.74,94.451098
5,AMT,270.70,73,45.83,0.52,22.12,0.82,12.18,0.72,26.702235,0.4,22.478272,0.9,93.572854
6,LLY,343.35,58,49.52,0.6,34.01,0.88,11.02,0.58,31.894079,0.62,14.428505,0.34,93.093812
7,ADSK,201.73,99,93.23,0.84,64.24,0.98,9.62,0.38,53.972446,0.96,10.420993,0.12,93.093812
8,ABMD,287.70,69,97.26,0.9,8.89,0.38,12.78,0.76,42.949156,0.88,14.585619,0.36,92.974052
9,MA,345.08,57,37.41,0.32,48.82,0.92,16.90,0.92,28.846039,0.52,16.908295,0.64,92.934132


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [130]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    # print(value_percentiles)
rv_dataframe

/Users/luxiaotong/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,451.65,44,411.08,1.0,24.16,0.86,15.37,0.86,125.928185,1.0,18.348054,0.72,0.888
1,PAYC,336.95,59,85.53,0.8,20.41,0.8,17.89,0.94,52.361429,0.94,18.870756,0.76,0.848
2,CDNS,171.51,116,64.84,0.76,17.54,0.73,15.23,0.84,50.793766,0.92,16.694200,0.6,0.77
3,NVDA,177.24,112,48.02,0.56,17.54,0.73,15.77,0.88,36.255665,0.78,23.216891,0.92,0.774
4,IDXX,411.82,48,47.74,0.54,54.10,0.94,10.73,0.54,33.139104,0.7,18.641545,0.74,0.692
5,AMT,270.70,73,45.83,0.52,22.12,0.82,12.18,0.72,26.702235,0.4,22.478272,0.9,0.672
6,LLY,343.35,58,49.52,0.6,34.01,0.88,11.02,0.58,31.894079,0.62,14.428505,0.34,0.604
7,ADSK,201.73,99,93.23,0.84,64.24,0.98,9.62,0.38,53.972446,0.96,10.420993,0.12,0.656
8,ABMD,287.70,69,97.26,0.9,8.89,0.38,12.78,0.76,42.949156,0.88,14.585619,0.36,0.656
9,MA,345.08,57,37.41,0.32,48.82,0.92,16.90,0.92,28.846039,0.52,16.908295,0.64,0.664


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [131]:
rv_dataframe.sort_values('RV Score', ascending=False, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

<ipython-input-131-73a3846cb171>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=False, inplace=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,451.65,44,411.08,1.0,24.16,0.86,15.37,0.86,125.928185,1.0,18.348054,0.72,0.888
1,PAYC,336.95,59,85.53,0.8,20.41,0.8,17.89,0.94,52.361429,0.94,18.870756,0.76,0.848
2,NVDA,177.24,112,48.02,0.56,17.54,0.73,15.77,0.88,36.255665,0.78,23.216891,0.92,0.774
3,CDNS,171.51,116,64.84,0.76,17.54,0.73,15.23,0.84,50.793766,0.92,16.694200,0.6,0.77
4,IDXX,411.82,48,47.74,0.54,54.10,0.94,10.73,0.54,33.139104,0.7,18.641545,0.74,0.692
5,AMT,270.70,73,45.83,0.52,22.12,0.82,12.18,0.72,26.702235,0.4,22.478272,0.9,0.672
6,MA,345.08,57,37.41,0.32,48.82,0.92,16.90,0.92,28.846039,0.52,16.908295,0.64,0.664
7,O,72.69,275,73.37,0.78,1.73,0.02,17.97,0.96,29.584527,0.56,24.088147,0.98,0.66
8,ADSK,201.73,99,93.23,0.84,64.24,0.98,9.62,0.38,53.972446,0.96,10.420993,0.12,0.656
9,ABMD,287.70,69,97.26,0.9,8.89,0.38,12.78,0.76,42.949156,0.88,14.585619,0.36,0.656


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [120]:
portfolio_input()

In [132]:
position_size = math.floor(float(portfolio_size)/len(rv_dataframe.index))
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,451.65,44,411.08,1.0,24.16,0.86,15.37,0.86,125.928185,1.0,18.348054,0.72,0.888
1,PAYC,336.95,59,85.53,0.8,20.41,0.8,17.89,0.94,52.361429,0.94,18.870756,0.76,0.848
2,NVDA,177.24,112,48.02,0.56,17.54,0.73,15.77,0.88,36.255665,0.78,23.216891,0.92,0.774
3,CDNS,171.51,116,64.84,0.76,17.54,0.73,15.23,0.84,50.793766,0.92,16.694200,0.6,0.77
4,IDXX,411.82,48,47.74,0.54,54.10,0.94,10.73,0.54,33.139104,0.7,18.641545,0.74,0.692
5,AMT,270.70,73,45.83,0.52,22.12,0.82,12.18,0.72,26.702235,0.4,22.478272,0.9,0.672
6,MA,345.08,57,37.41,0.32,48.82,0.92,16.90,0.92,28.846039,0.52,16.908295,0.64,0.664
7,O,72.69,275,73.37,0.78,1.73,0.02,17.97,0.96,29.584527,0.56,24.088147,0.98,0.66
8,ADSK,201.73,99,93.23,0.84,64.24,0.98,9.62,0.38,53.972446,0.96,10.420993,0.12,0.656
9,ABMD,287.70,69,97.26,0.9,8.89,0.38,12.78,0.76,42.949156,0.88,14.585619,0.36,0.656


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [133]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, 'Value Strategy', index=False)


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [134]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    ) 

In [135]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }
for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [136]:
writer.save()